# 情報システム工学演習 画像処理 OpenCVチュートリアル

OpenCVを使った画像処理の演習です。まずはこのチュートリアルで基礎の基礎を学んでいきましょう。
ちなみに、OpenCVの公式チュートリアルが https://docs.opencv.org/4.x/d9/df8/tutorial_root.html にあるので、こちらも活用してください。

## 準備
インストールについては```README.md```を参照してください。

In [ ]:
import cv2
import numpy as np  # PythonのOpenCVでは、画像はnumpyのarrayとして管理される
from PIL import Image
import matplotlib.pyplot as plt
from IPython.display import display
%matplotlib inline

# 画像表示用の関数（jupyter notebookでインラインで表示したい時向け）
def imshow_inline(img):
  if img.ndim == 3:
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB) # OpenCVの画像は、BGRの順に色が並んでいるので
    display(Image.fromarray(img))
  else:
    display(Image.fromarray(img))

## 画像の読み込み・書き込み編

In [ ]:
# 画像ファイルの読み込み・書き込み
img = cv2.imread("sample/sample.jpg")
# img = cv2.imread("sample/sample.jpg",cv2.IMREAD_GRAYSCALE)  # 強制的にグレースケール（白黒画像）として読み込む場合

# てきとうな処理（リサイズしてみた）
#dst = cv2.resize(img,dsize=None,fx=0.5,fy=0.5)  # 縦横半分
dst = cv2.resize(img,dsize=(512,256))  # 指定したサイズ

# 画像の書き込み
cv2.imwrite("out.png",dst)

# インライン表示する場合
#imshow_inline(img)
#imshow_inline(dst)

# 新しいウインドウを開いて表示する場合（ウインドウを閉じるか、なにかキーを押すと終了）
cv2.namedWindow('src') # 指定されたタイトルのウインドウを開く
cv2.imshow('src',img)  # 指定されたタイトルのウインドウに画像を表示
cv2.namedWindow('dst') # 指定されたタイトルのウインドウを開く
cv2.imshow('dst',dst)  # 指定されたタイトルのウインドウに画像を表示
cv2.waitKey(0)           # キーが押されるまで{引数}[ms]の間待つ（0の場合はずっと待つ）
cv2.destroyAllWindows()  # ウインドウを閉じる。!!!jupyter上の場合、これを忘れるとフリーズする!!!

In [ ]:
# 動画の読み込み
cap = cv2.VideoCapture('sample/sample.avi')

end_flag, frame = cap.read()  # 最初のフレームを読み込み
height, width, channels = frame.shape

cv2.namedWindow('image') # 'image'というタイトルのウインドウを開く

while end_flag == True:
  cv2.imshow('image',frame)  # 'image'というタイトルのウインドウに画像を表示

  key = cv2.waitKey(30) # キーが押されるまで{引数}[ms]の間待つ（0の場合はずっと待つ）
  if key == 27 or key == ord('q'):  # qかESCが押されたら途中で終了
    break

  end_flag, frame = cap.read() # 次のフレームを読み込み

cv2.destroyAllWindows()  # ウインドウを閉じる
cap.release()

In [ ]:
# webカメラの読み込み
# !!!注意!!! 当然ですが、カメラがつながっていないとエラーになります

# 引数はデバイスの順番。複数のカメラがつながっている場合は1などに変えると変わる
cap = cv2.VideoCapture(0)

end_flag, frame = cap.read()  # 最初のフレームを読み込み
height, width, channels = frame.shape

cv2.namedWindow('image') # 'image'というタイトルのウインドウを開く

while end_flag == True:
  cv2.imshow('image',frame)  # 'image'というタイトルのウインドウに画像を表示

  key = cv2.waitKey(30) # キーが押されるまで{引数}[ms]の間待つ（0の場合はずっと待つ）
  if key == 27 or key == ord('q'):  # qかESCが押されたら途中で終了
    break

  end_flag, frame = cap.read() # 次のフレームを読み込み

cv2.destroyAllWindows()  # ウインドウを閉じる
cap.release()


## 画素へのアクセスと色の操作編

Tips:
- 画像は配列。画像のサイズを縦h, 横w, チャンネル数cで表すと、
  - グレースケール（1ch）画像の場合、(h,w)の2次元のnumpy配列
  - カラー画像（3ch, BGRの順で色が並ぶ）の場合、(h,w,c)の3次元のnumpy配列
    - カラー画像の画素のチャンネルは、それぞれ青・緑・赤の色の強さを表す。
- 原点は左上。縦軸をy、横軸をxとすると、左からx、上からy画素目の画素へのアクセスは`img[y][x] (img[y,x]でもOK)`（1ch画像の場合）
- 配列の各要素には0~255の8bit (uint8)が格納されており、大きい値が明るい（あるいは色が強い）画素を表す。
- 画素へのアクセスや操作は、for文の2重ループ（それぞれ縦横の座標を表す）で走査する方法がある
  - Pythonのfor文は遅いので、numpyやOpenCVの関数を使った効率化を積極的に検討すること。

In [ ]:
# グレースケール画像の操作
img_color = cv2.imread("sample/sample.jpg")
print("color (BGR): ",img_color.shape, ", type: ", img_color.dtype)

img_gray = cv2.cvtColor(img_color,cv2.COLOR_BGR2GRAY) # グレイスケールに変換
print("grayscale: ",img_gray.shape, ", type: ", img_color.dtype)

# 各画素をスキャンして、条件にあった画素だけ書き換える（効率悪い。おそい。）
out1 = img_gray.copy()
out2 = img_gray.copy()
for y in range(img_gray.shape[0]):
  for x in range(img_gray.shape[1]):

    # 例1: 画素値が128よりも小さい場合に全部255（白）にする
    if out1[y,x] < 128:
      out1[y,x] = 255

    # 例2: y座標が30以上110未満の部分を黒塗り
    if 30 <= y < 110:
      out2[y,x] = 0

# 例1は、numpyを使って以下のようにも実現できる。はやい。
out_numpy = np.where(img_gray<128, 255, img_gray)
# 例2は、numpyの配列操作で以下のように書ける。はやい。
out2_numpy = img_gray.copy()
out2_numpy[30:110,:] = 0

imshow_inline(img_gray)
imshow_inline(out1)
imshow_inline(out2)

In [ ]:
# カラー画像の操作
img = cv2.imread("sample/sample.jpg")

out = img.copy()
for y in range(img.shape[0]):
  for x in range(img.shape[1]):

    # R（赤成分）だけ大きい場合、黒(0,0,0)にする
    if img[y,x,0] < 128 and img[y,x,1] < 128 and img[y,x,2] > 128:
      out[y,x,0] = 0
      out[y,x,1] = 0
      out[y,x,2] = 0

# OpenCVとnumpyを使って以下のようにも書ける
out_numpy = img.copy()
mask = cv2.inRange(img, np.array([0,0,128]), np.array([128,128,255]))
mask = 255-mask # Rだけ大きい画素「以外」を残したいので
mask = np.dstack((mask,mask,mask))  # 3chにしたいので
out_numpy = cv2.bitwise_and(img, mask)
#out_numpy = img * (mask/255).astype(np.uint8)

imshow_inline(img)
imshow_inline(out)
imshow_inline(out_numpy)

## 画像の統計量

画像のヒストグラム（度数分布）を計算するなどが代表的。

In [ ]:
img = cv2.imread("sample/sample.jpg")

hist_B = cv2.calcHist([img],channels=[0],mask=None,histSize=[256],ranges=[0,256])  # B:0, G:1, R:2
hist_G = cv2.calcHist([img],channels=[1],mask=None,histSize=[256],ranges=[0,256])  # B:0, G:1, R:2
hist_R = cv2.calcHist([img],channels=[2],mask=None,histSize=[256],ranges=[0,256])  # B:0, G:1, R:2

fig, ax = plt.subplots()
x = range(0,256)
x_axis = np.array(x)
ax.plot(x_axis, np.array(hist_B), label='B', color='blue')
ax.plot(x_axis, np.array(hist_G), label='G', color='green')
ax.plot(x_axis, np.array(hist_R), label='R', color='red')
ax.legend(loc=0)    # 凡例

## 画像のフィルタリング

画像の編集や特徴の抽出に使える。

In [ ]:
# 画像を「ぼかす」

img = cv2.imread("sample/wani.png")
dst = cv2.GaussianBlur(img,ksize=(21,21),sigmaX=7)

print("input")
imshow_inline(img)

print("blurred")
imshow_inline(dst)

In [ ]:
# 画像から「エッジ」を抽出する

img = cv2.imread("sample/prims.png")
img_gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) # グレイスケールに変換

grad_x = cv2.Sobel(img_gray,cv2.CV_64F,1,0,ksize=5) # x方向の変化の大きな点（縦方向のエッジ）を検出
grad_y = cv2.Sobel(img_gray,cv2.CV_64F,0,1,ksize=5) # y方向の変化の大きな点（横方向のエッジ）を検出

# 勾配（画素値の変化）の強さと方向を計算する。angle: [0,360)
magnitude, angle = cv2.cartToPolar(grad_x, grad_y,angleInDegrees=True)

fig, ax = plt.subplots()
im = ax.imshow(magnitude)
fig.colorbar(im, ax=ax)
plt.show()

fig, ax = plt.subplots()
im = ax.imshow(angle)
fig.colorbar(im, ax=ax)
plt.show()

## 画像の変形編

座標(x,y)を操作することで、画像を変形（ワーピング）することができる。詳しくは、射影変換（ホモグラフィ変換）を調べてみよう

以下は、2つの画像を張り合わせる例。4組の対応点を指定し、その前後の変換（ホモグラフィ行列）を計算して変形する。

In [ ]:
# imgをrefに張り合わせることを考える
ref = cv2.imread("sample/pano_ref.jpg") # ベースとなる画像（BGR）
src = cv2.imread("sample/pano_src.jpg") # 変換する画像（BGR）

print("ref")
imshow_inline(ref)
print("src")
imshow_inline(src)

pts_ref = np.float32([[923,156],[1281,143],[1276,760],[916,745]]) # refの点(x', y')
pts_src = np.float32([[88,163],[438,190],[437,760],[78,782]])  # srcの点(x, y)

# 横幅2倍の画像を作って、そこに貼り付けることにした。配列作成時のshapeの設定はw,h,cの順なので注意
dst = np.zeros((src.shape[0],src.shape[1]*2,3))
dst[:,0:ref.shape[1],:] = ref # 左半分にrefをコピー
H = cv2.getPerspectiveTransform(pts_src,pts_ref)  # 最小二乗法によるホモグラフィ行列の推定（img -> refへの変換）
src_warped = cv2.warpPerspective(src, H, (src.shape[1]*2,src.shape[0]), flags=cv2.INTER_NEAREST) # ホモグラフィ行列を使ったワーピング

mask = np.where(src_warped==0,0,1).astype(np.uint8) # 変換後srcの値がある場所が1になるマスク画像を作る
dst = dst*(1-mask) + src_warped*mask  # アルファブレンディング
dst = dst.astype(np.uint8)  # uint8に変換

print("dst")
imshow_inline(dst)



## 画像の対応付け

画像の同じ部分はどこか？を探したい。そんな時は対応付けが便利。
画像上の特徴的な点を探し、それら同士のマッチングを行っている。

https://docs.opencv.org/4.x/db/d27/tutorial_py_table_of_contents_feature2d.html

In [ ]:
# imgとref間のマッチングをする
ref = cv2.imread("sample/pano_ref.jpg")
src = cv2.imread("sample/pano_src.jpg")

# ここでは、ORBという特徴点・特徴量を使ってみる。
orb = cv2.ORB_create()

# 各画像に対する特徴点（keypoint）とそれに付随する特徴量（descriptor）の計算
kp_ref, des_ref = orb.detectAndCompute(ref, None)
kp_src, des_src = orb.detectAndCompute(src, None)

# マッチング。特徴量の性質から、ハミング距離を使う。
matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = matcher.match(des_ref,des_src)

# 全対応を可視化。対応づいた点同士が線で結ばれる。
corr_disp = cv2.drawMatches(ref,kp_ref,src,kp_src,matches,None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
imshow_inline(corr_disp) # 表示

## ユーザ入力の取得・描画
OpenCVでは、namedWindow上でのマウスやキーボード入力の取得が可能

また、画像上への描画関数も実装されている。丸以外に、直線（```cv2.line```）、長方形（```cv2.rectangle```）なども。

In [ ]:
img = cv2.imread("sample/sample.jpg")
draw = img.copy()

# マウスコールバック関数。マウスイベントが発生すると呼ばれる
def mouse_callback(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        cv2.circle(draw,(x,y),10,(0,255,0), 2)
        print("left press: (", x, ",", y, ")")
    if event == cv2.EVENT_RBUTTONDOWN:
        cv2.circle(draw,(x,y),10,(255,0,0), 2)
        print("right press: (", x, ",", y, ")")

cv2.namedWindow("window")
cv2.setMouseCallback("window", mouse_callback)  # マウスコールバック関数の設定

key = 0
while True:
  cv2.imshow('window',draw)  # 'image'というタイトルのウインドウに画像を表示

  # キーボード入力はcv2.waitKeyで。
  key = cv2.waitKey(30) # キーが押されるまで{引数}[ms]の間待つ（0の場合はずっと待つ）
  if key == 27 or key == ord('q'):  # qかESCが押されたら途中で終了
    break

cv2.destroyAllWindows()  # ウインドウを閉じる


## 物体検出
ここでは、Haar-Cascade特徴を使った顔検出を紹介する。それ以外の学習済み特徴も、`haarcascades`ディレクトリ以下にある。目鼻口なども。

https://docs.opencv.org/4.x/db/d28/tutorial_cascade_classifier.html

In [ ]:
# 学習済みファイルの入力
face_cascade_file  = "haarcascades/haarcascade_frontalface_default.xml"
face_cascade = cv2.CascadeClassifier(face_cascade_file)

img = cv2.imread("sample/face.png")
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

face_list = face_cascade.detectMultiScale(img_gray, minSize=(30, 30))   # 顔検出

for (x, y, w, h) in face_list:  # 検出した顔
    #print(face_list)
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 225), thickness = 3)  # 顔の枠を描く

imshow_inline(img) # 表示